In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil
from tifffile import imread, imwrite

In [2]:
filemap_path = "/mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/analysis/report/analysis_filemap_annotated.csv"
filemap = pd.read_csv(filemap_path)

db_path = "/mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db_2/"
os.makedirs(db_path, exist_ok=True)
os.makedirs(os.path.join(db_path, "img_ch1"), exist_ok=True)
os.makedirs(os.path.join(db_path, "img_ch2"), exist_ok=True)
os.makedirs(os.path.join(db_path, "label"), exist_ok=True)

In [3]:
def pick_within_larval_stage(stacks, stacks_time, ls_beg, ls_end):
    if np.isnan(ls_beg) or np.isnan(ls_end):
        return None, None

    valid_stacks = [(s, t) for s, t in zip(stacks, stacks_time) if ls_beg <= t <= ls_end]
    if valid_stacks:
        random_stack, random_stack_time = valid_stacks[np.random.randint(len(valid_stacks))]
        return random_stack, random_stack_time
    else:
        return None, None


for point in filemap['Point'].unique():
    point_df = filemap[filemap['Point'] == point]
    hatch_time, m1, m2, m3= point_df['HatchTime'].values[0], point_df['M1'].values[0], point_df['M2'].values[0], point_df['M3'].values[0]

    if np.isnan(hatch_time):
        hatch_time = 0.
    print(f"Point {point}: Hatch Time: {hatch_time}, M1: {m1}, M2: {m2}, M3: {m3}")

    raw_images = point_df['raw'].values
    time = point_df['Time'].values

    stacks_idx = [i for i, img_path in enumerate(raw_images) if "DIA" not in img_path]
    stacks = [raw_images[i] for i in stacks_idx]
    stacks_time = [time[i] for i in stacks_idx]

    for i, (ls_beg, ls_end) in enumerate([(hatch_time, m1), (m1, m2), (m2, m3)]):
        random_stack, random_stack_time = pick_within_larval_stage(stacks, stacks_time, ls_beg, ls_end)

        if random_stack:
            stack = imread(random_stack)
            fake_mask = np.zeros_like(stack)[:, 0]
            filename = f"Time{str(i).zfill(5)}_Point{str(point).zfill(4)}.ome.tiff"
            imwrite(os.path.join(db_path, 'img_ch1', filename), stack[:, 0], compression='zlib')
            imwrite(os.path.join(db_path, 'img_ch2', filename), stack[:, 1], compression='zlib')
            imwrite(os.path.join(db_path, 'label', filename), fake_mask, compression='zlib')
        else:
            print(f"L{i+1}: No stack found within the larval stage")

Point 0: Hatch Time: 0.0, M1: 71.0, M2: 121.0, M3: 170.0
Point 1: Hatch Time: 19.0, M1: 94.0, M2: 149.0, M3: 200.0
Point 2: Hatch Time: 7.0, M1: 86.0, M2: 143.0, M3: 187.0
Point 3: Hatch Time: 16.0, M1: 97.0, M2: 149.0, M3: 194.0
Point 4: Hatch Time: 7.0, M1: 82.0, M2: 131.0, M3: 182.0
Point 5: Hatch Time: 19.0, M1: nan, M2: nan, M3: nan
L1: No stack found within the larval stage
L2: No stack found within the larval stage
L3: No stack found within the larval stage
Point 6: Hatch Time: 0.0, M1: 79.0, M2: 140.0, M3: 191.0
Point 7: Hatch Time: 7.0, M1: 88.0, M2: 137.0, M3: 187.0
Point 8: Hatch Time: 23.0, M1: 101.0, M2: 151.0, M3: 200.0
Point 9: Hatch Time: 0.0, M1: 77.0, M2: 128.0, M3: 179.0
Point 10: Hatch Time: 25.0, M1: 107.0, M2: 173.0, M3: 218.0
Point 11: Hatch Time: 4.0, M1: 80.0, M2: 128.0, M3: 173.0
Point 12: Hatch Time: 29.0, M1: 122.0, M2: 179.0, M3: nan
L3: No stack found within the larval stage
Point 13: Hatch Time: 16.0, M1: 95.0, M2: 146.0, M3: 196.0
Point 14: Hatch Time: 2